# Music Genre Classification Using Audio Feature Extraction and Machine-Learning Models

**Course:** CAP 4630-001-042 - Introduction to Artificial Intelligence  
**Instructor:** Dr. Ahmed Imteaj  
**Student:** Andres Hernandez  
**Date:** Fall 2025

---

## Project Description

This project implements a supervised machine learning system for classifying audio tracks into musical genres using Mel-Frequency Cepstral Coefficients (MFCCs) as the primary feature representation. The system extracts audio features from raw audio files, processes them into fixed-length vectors, and trains multiple classification models to predict genre labels.

## Deliverables

1. Complete audio feature extraction pipeline using librosa
2. Implementation of multiple baseline classifiers (KNN, Decision Trees, Random Forest, SVM)
3. Hyperparameter tuning and model optimization
4. Comprehensive evaluation with confusion matrices and performance metrics
5. Analysis of results, challenges, and future improvements

---

## 1. Objective

### Task Definition
Classify audio tracks into musical genres using MFCC-based audio features and supervised machine learning classifiers.

### Target Outcome
Develop a trained classification model with measurable performance metrics (accuracy, precision, recall, F1-score) that can accurately predict the genre of previously unseen audio tracks.

### Success Criteria
- Achieve baseline accuracy above random guessing (>10% for 10 genres)
- Compare multiple classification algorithms
- Identify optimal hyperparameters through systematic tuning
- Document model performance with comprehensive evaluation metrics

---

## 2. Background

### Why Music Genre Classification?

Music genre classification is a fundamental task in Music Information Retrieval (MIR) with practical applications in:
- **Music streaming services**: Automated playlist generation and recommendation systems
- **Music libraries**: Organizing and categorizing large audio collections
- **Music production**: Assisting composers and producers in understanding musical patterns
- **Academic research**: Understanding the structural and timbral characteristics that define genres

### Mel-Frequency Cepstral Coefficients (MFCCs)

MFCCs are the standard audio feature representation for speech and music analysis because they:
- Model the human auditory system's perception of sound
- Capture the spectral envelope of audio signals
- Provide compact, fixed-length representations regardless of audio duration
- Effectively represent timbral texture, which is crucial for genre differentiation

**Technical Process:**
1. Convert time-domain audio to frequency domain via Short-Time Fourier Transform (STFT)
2. Apply mel-scale filterbank to match human perception
3. Take logarithm of filterbank energies
4. Apply Discrete Cosine Transform (DCT) to decorrelate features
5. Extract lower-order coefficients (typically 13-40) as features

### Classification Algorithms

For structured numeric features like MFCCs, several supervised classifiers are suitable:
- **K-Nearest Neighbors (KNN)**: Instance-based learning, effective for multi-class problems
- **Decision Trees**: Interpretable, handles non-linear decision boundaries
- **Random Forest**: Ensemble method, reduces overfitting through averaging
- **Support Vector Machines (SVM)**: Effective in high-dimensional spaces with clear margins

---

## 3. Setup and Imports

Import all necessary libraries for audio processing, machine learning, and visualization.

In [1]:
# Standard libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# Audio processing
import librosa
import librosa.display

# Machine learning
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score,
    confusion_matrix,
    classification_report
)

# Settings
warnings.filterwarnings('ignore')
np.random.seed(42)
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)

print("All libraries imported successfully!")

All libraries imported successfully!


---

## Quick Start Guide

**To run this notebook:**

1. **Run the imports cell above** ☝️ to load all libraries
2. **Run the dataset configuration cell below** 👇 to verify GTZAN path
3. **Execute cells sequentially** - each section builds on the previous one
4. **Feature extraction takes 10-20 minutes** - be patient!
5. **Watch for progress indicators** in the output

**Current Dataset Path:** `/Users/andres/Documents/University/FALL 2025 /CAP4630/Dataset/GTZAN/genres_original`

✅ Dataset is already configured and ready to use!

---

## 4. Dataset Description

### Dataset Source
**GTZAN Genre Collection** (or specify your actual dataset)

### Dataset Specifications
- **Number of samples**: 1000 audio files (100 per genre)
- **Genres**: 10 classes (blues, classical, country, disco, hiphop, jazz, metal, pop, reggae, rock)
- **File format**: .wav or .mp3
- **Duration**: 30 seconds per track
- **Sampling rate**: 22,050 Hz (standard for music analysis)
- **Class balance**: Balanced dataset with equal samples per genre

### Preprocessing Requirements
- Handle corrupted or unreadable audio files
- Normalize audio length to consistent duration
- Convert stereo to mono if necessary
- Resample to target sampling rate if needed

In [2]:
# Dataset configuration
DATASET_PATH = "/Users/andres/Documents/University/FALL 2025 /CAP4630/Dataset/GTZAN/genres_original"
SAMPLING_RATE = 22050
DURATION = 30  # seconds
N_MFCC = 13  # Number of MFCC coefficients to extract

# Check if dataset path exists
if os.path.exists(DATASET_PATH):
    print(f"✓ Dataset found at: {DATASET_PATH}")
    
    # Count files per genre
    genres = sorted([d for d in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, d))])
    print(f"\nGenres found: {len(genres)}")
    for genre in genres:
        genre_path = os.path.join(DATASET_PATH, genre)
        wav_files = [f for f in os.listdir(genre_path) if f.endswith('.wav')]
        print(f"  {genre}: {len(wav_files)} files")
else:
    print(f"⚠️ Dataset path not found. Please update DATASET_PATH variable.")
    print(f"Current path: {DATASET_PATH}")

✓ Dataset found at: /Users/andres/Documents/University/FALL 2025 /CAP4630/Dataset/GTZAN/genres_original

Genres found: 10
  blues: 100 files
  classical: 100 files
  country: 100 files
  disco: 100 files
  hiphop: 100 files
  jazz: 100 files
  metal: 100 files
  pop: 100 files
  reggae: 100 files
  rock: 100 files


---

## 5. Methodology

### Complete Pipeline (Sequential Steps)

1. **Load audio**: Read audio files at fixed sampling rate (22,050 Hz)
2. **Extract MFCCs**: Compute MFCC features with fixed parameters (n_mfcc=13)
3. **Aggregate features**: Convert variable-length MFCC matrices into fixed-length vectors using mean/std
4. **Build feature matrix**: Combine all feature vectors and labels into structured arrays
5. **Split dataset**: Divide into training (80%) and test (20%) sets with stratification
6. **Train baseline models**: Fit KNN, Decision Tree, Random Forest, and SVM classifiers
7. **Hyperparameter tuning**: Use GridSearchCV to optimize model parameters
8. **Evaluate models**: Compute accuracy, precision, recall, F1-score, and confusion matrices

### Technical Constraints
- Fixed MFCC vector dimension: 26 features (13 mean + 13 std coefficients)
- Consistent sampling rate across all audio files
- Stratified splitting to maintain class balance

---

## 6. System Design

### Pipeline Architecture

```
┌─────────────┐      ┌──────────────────┐      ┌────────────────────┐
│ Input Audio │ ───> │ Feature          │ ───> │ Feature            │
│ (.wav/.mp3) │      │ Extraction       │      │ Aggregation        │
│             │      │ (MFCC)           │      │ (Mean + Std)       │
└─────────────┘      └──────────────────┘      └────────────────────┘
                              │                          │
                              ▼                          ▼
                     22050 Hz audio          [n_samples × 26] matrix
                     13 MFCC coefficients    (13 mean + 13 std)
                              │                          │
                              └──────────┬───────────────┘
                                         ▼
                              ┌────────────────────┐
                              │ Train/Test Split   │
                              │ (80/20)            │
                              └────────────────────┘
                                         │
                              ┌──────────┴──────────┐
                              ▼                     ▼
                     ┌──────────────┐      ┌──────────────┐
                     │ Model        │      │ Model        │
                     │ Training     │      │ Evaluation   │
                     └──────────────┘      └──────────────┘
                              │                     │
                              ▼                     ▼
                     ┌──────────────┐      ┌──────────────┐
                     │ Trained      │      │ Performance  │
                     │ Classifier   │      │ Metrics      │
                     └──────────────┘      └──────────────┘
                              │                     │
                              └──────────┬──────────┘
                                         ▼
                              ┌────────────────────┐
                              │ Genre Prediction   │
                              └────────────────────┘
```

---

## 7. Implementation

### Part A: Data Loading and Audio Preprocessing

Load audio files from the dataset directory, handle corrupted files, and organize data by genre labels.

In [ ]:
def load_audio_files(dataset_path):
    """
    Load audio files from dataset directory and extract genre labels.
    
    Parameters:
    -----------
    dataset_path : str
        Path to the dataset root directory
        
    Returns:
    --------
    audio_data : list
        List of tuples (file_path, genre_label)
    """
    audio_data = []
    corrupted_files = []
    
    # Assuming dataset structure: dataset_path/genre/audio_files
    if not os.path.exists(dataset_path):
        print(f"Dataset path does not exist: {dataset_path}")
        return audio_data, corrupted_files
    
    # Iterate through genre folders
    for genre in os.listdir(dataset_path):
        genre_path = os.path.join(dataset_path, genre)
        
        # Skip if not a directory
        if not os.path.isdir(genre_path):
            continue
            
        print(f"Loading {genre} files...")
        
        # Iterate through audio files in genre folder
        for filename in os.listdir(genre_path):
            file_path = os.path.join(genre_path, filename)
            
            # Check if it's an audio file
            if filename.endswith(('.wav', '.mp3', '.au')):
                try:
                    # Test loading the file
                    y, sr = librosa.load(file_path, sr=SAMPLING_RATE, duration=DURATION)
                    audio_data.append((file_path, genre))
                except Exception as e:
                    corrupted_files.append((file_path, str(e)))
                    print(f"  ⚠️ Corrupted file: {filename}")
    
    print(f"\nSuccessfully loaded {len(audio_data)} audio files")
    print(f"Corrupted files: {len(corrupted_files)}")
    
    return audio_data, corrupted_files

# Load dataset
print("="*70)
print("LOADING GTZAN DATASET")
print("="*70)
audio_files, corrupted = load_audio_files(DATASET_PATH)

### Part B: MFCC Feature Extraction

Extract MFCC features from audio files and aggregate them into fixed-length feature vectors.

In [ ]:
def extract_mfcc_features(file_path, n_mfcc=13, sr=22050, duration=30):
    """
    Extract MFCC features from an audio file.
    
    Parameters:
    -----------
    file_path : str
        Path to the audio file
    n_mfcc : int
        Number of MFCC coefficients to extract
    sr : int
        Target sampling rate
    duration : int
        Duration in seconds to load
        
    Returns:
    --------
    mfcc : ndarray
        MFCC feature matrix of shape (n_mfcc, n_frames)
    """
    try:
        # Load audio file
        y, sr = librosa.load(file_path, sr=sr, duration=duration)
        
        # Extract MFCCs
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        
        return mfcc
    except Exception as e:
        print(f"Error extracting features from {file_path}: {e}")
        return None


def aggregate_mfcc_features(mfcc):
    """
    Aggregate MFCC matrix into fixed-length feature vector using mean and std.
    
    Parameters:
    -----------
    mfcc : ndarray
        MFCC feature matrix of shape (n_mfcc, n_frames)
        
    Returns:
    --------
    features : ndarray
        Aggregated feature vector of shape (n_mfcc * 2,)
    """
    # Calculate mean and standard deviation across time axis
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    
    # Concatenate mean and std to create fixed-length feature vector
    features = np.concatenate([mfcc_mean, mfcc_std])
    
    return features


def extract_features_from_dataset(audio_files, n_mfcc=13):
    """
    Extract features from all audio files in the dataset.
    
    Parameters:
    -----------
    audio_files : list
        List of tuples (file_path, genre_label)
    n_mfcc : int
        Number of MFCC coefficients
        
    Returns:
    --------
    X : ndarray
        Feature matrix of shape (n_samples, n_features)
    y : list
        List of genre labels
    """
    X = []
    y = []
    
    print(f"Extracting features from {len(audio_files)} files...")
    
    for i, (file_path, genre) in enumerate(audio_files):
        # Extract MFCCs
        mfcc = extract_mfcc_features(file_path, n_mfcc=n_mfcc)
        
        if mfcc is not None:
            # Aggregate features
            features = aggregate_mfcc_features(mfcc)
            X.append(features)
            y.append(genre)
        
        # Progress indicator
        if (i + 1) % 100 == 0:
            print(f"  Processed {i + 1}/{len(audio_files)} files")
    
    X = np.array(X)
    print(f"\nFeature extraction complete!")
    print(f"Feature matrix shape: {X.shape}")
    print(f"Number of samples: {len(y)}")
    
    return X, y

# Extract features from all audio files
print("\n" + "="*70)
print("EXTRACTING MFCC FEATURES")
print("="*70)
X, y = extract_features_from_dataset(audio_files, n_mfcc=N_MFCC)

### Part C: Visualize Sample MFCC Features

Visualize MFCCs from a sample audio file to understand the feature representation.

In [ ]:
def visualize_mfcc(file_path, genre):
    """
    Visualize MFCC features for a sample audio file.
    """
    # Load audio
    y, sr = librosa.load(file_path, sr=SAMPLING_RATE, duration=DURATION)
    
    # Extract MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC)
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Plot waveform
    librosa.display.waveshow(y, sr=sr, ax=axes[0])
    axes[0].set_title(f'Waveform - {genre}', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')
    axes[0].grid(True, alpha=0.3)
    
    # Plot MFCCs
    img = librosa.display.specshow(mfcc, x_axis='time', ax=axes[1], cmap='viridis')
    axes[1].set_title(f'MFCC Features - {genre}', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('MFCC Coefficients')
    axes[1].set_xlabel('Time (s)')
    fig.colorbar(img, ax=axes[1], format='%+2.0f')
    
    plt.tight_layout()
    plt.show()
    
    # Print aggregated features
    features = aggregate_mfcc_features(mfcc)
    print(f"Aggregated feature vector shape: {features.shape}")
    print(f"First 10 features (mean of first 10 MFCCs): {features[:10]}")

# Visualize sample MFCC
# Uncomment and update with actual file path
# sample_file = audio_files[0][0]
# sample_genre = audio_files[0][1]
# visualize_mfcc(sample_file, sample_genre)

### Part D: Exploratory Data Analysis

Analyze the dataset distribution and feature characteristics.

In [ ]:
# Create DataFrame for analysis
# Uncomment after extracting features
# df = pd.DataFrame(X)
# df['genre'] = y

# print("Dataset Summary:")
# print(f"Total samples: {len(df)}")
# print(f"\nGenre distribution:")
# print(df['genre'].value_counts().sort_index())
# print(f"\nFeature statistics:")
# print(df.drop('genre', axis=1).describe())

# # Visualize genre distribution
# plt.figure(figsize=(10, 6))
# df['genre'].value_counts().sort_index().plot(kind='bar', color='steelblue')
# plt.title('Genre Distribution in Dataset', fontsize=14, fontweight='bold')
# plt.xlabel('Genre')
# plt.ylabel('Number of Samples')
# plt.xticks(rotation=45)
# plt.grid(axis='y', alpha=0.3)
# plt.tight_layout()
# plt.show()

### Part E: Data Preprocessing and Train/Test Split

Encode labels, standardize features, and split into training and test sets.

In [ ]:
# Encode genre labels
# Uncomment after feature extraction
# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(y)

# print(f"Label encoding:")
# for i, genre in enumerate(label_encoder.classes_):
#     print(f"  {genre}: {i}")

# # Split dataset (80% train, 20% test) with stratification
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y_encoded, 
#     test_size=0.2, 
#     random_state=42, 
#     stratify=y_encoded
# )

# print(f"\nDataset split:")
# print(f"Training samples: {len(X_train)}")
# print(f"Test samples: {len(X_test)}")
# print(f"Training set shape: {X_train.shape}")
# print(f"Test set shape: {X_test.shape}")

# # Standardize features
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# print(f"\nFeature standardization complete!")
# print(f"Training mean: {X_train_scaled.mean():.6f}")
# print(f"Training std: {X_train_scaled.std():.6f}")

---

## 8. Model Training and Evaluation

### Part A: Baseline Model Training

Train four baseline classifiers with default parameters.

In [ ]:
def train_baseline_models(X_train, y_train):
    """
    Train baseline models with default parameters.
    
    Returns:
    --------
    models : dict
        Dictionary of trained models
    """
    models = {
        'KNN': KNeighborsClassifier(),
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42),
        'SVM': SVC(random_state=42)
    }
    
    print("Training baseline models...\n")
    
    for name, model in models.items():
        print(f"Training {name}...")
        model.fit(X_train, y_train)
        print(f"  ✓ {name} training complete")
    
    print("\nAll baseline models trained successfully!")
    return models


def evaluate_model(model, X_test, y_test, model_name):
    """
    Evaluate a trained model and print performance metrics.
    """
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    print(f"\n{'='*50}")
    print(f"{model_name} Performance")
    print(f"{'='*50}")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'predictions': y_pred
    }

# Train baseline models
# Uncomment after data preprocessing
# baseline_models = train_baseline_models(X_train_scaled, y_train)

### Part B: Evaluate Baseline Models

Evaluate all baseline models on the test set.

In [ ]:
# Evaluate all baseline models
# Uncomment after training models
# results = {}
# for name, model in baseline_models.items():
#     results[name] = evaluate_model(model, X_test_scaled, y_test, name)

# # Create results DataFrame for comparison
# results_df = pd.DataFrame(results).T
# results_df = results_df.drop('predictions', axis=1)

# print("\n" + "="*70)
# print("BASELINE MODEL COMPARISON")
# print("="*70)
# print(results_df.round(4))

# # Visualize model comparison
# fig, axes = plt.subplots(2, 2, figsize=(14, 10))
# metrics = ['accuracy', 'precision', 'recall', 'f1_score']
# titles = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

# for idx, (metric, title) in enumerate(zip(metrics, titles)):
#     ax = axes[idx // 2, idx % 2]
#     results_df[metric].plot(kind='bar', ax=ax, color='steelblue')
#     ax.set_title(f'{title} Comparison', fontsize=12, fontweight='bold')
#     ax.set_ylabel(title)
#     ax.set_xlabel('Model')
#     ax.set_ylim([0, 1])
#     ax.grid(axis='y', alpha=0.3)
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# plt.tight_layout()
# plt.show()

### Part C: Hyperparameter Tuning

Optimize model hyperparameters using GridSearchCV.

In [ ]:
def hyperparameter_tuning(X_train, y_train):
    """
    Perform hyperparameter tuning for each model using GridSearchCV.
    
    Returns:
    --------
    tuned_models : dict
        Dictionary of tuned models with best parameters
    """
    # Define parameter grids for each model
    param_grids = {
        'KNN': {
            'n_neighbors': [3, 5, 7, 9, 11],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']
        },
        'Decision Tree': {
            'max_depth': [5, 10, 15, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        },
        'Random Forest': {
            'n_estimators': [50, 100, 200],
            'max_depth': [10, 20, None],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2]
        },
        'SVM': {
            'C': [0.1, 1, 10],
            'kernel': ['rbf', 'linear'],
            'gamma': ['scale', 'auto']
        }
    }
    
    base_models = {
        'KNN': KNeighborsClassifier(),
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42),
        'SVM': SVC(random_state=42)
    }
    
    tuned_models = {}
    best_params = {}
    
    print("Performing hyperparameter tuning...\n")
    
    for name in base_models.keys():
        print(f"Tuning {name}...")
        grid_search = GridSearchCV(
            base_models[name],
            param_grids[name],
            cv=5,
            scoring='accuracy',
            n_jobs=-1,
            verbose=0
        )
        
        grid_search.fit(X_train, y_train)
        tuned_models[name] = grid_search.best_estimator_
        best_params[name] = grid_search.best_params_
        
        print(f"  ✓ Best parameters: {grid_search.best_params_}")
        print(f"  ✓ Best CV score: {grid_search.best_score_:.4f}\n")
    
    print("Hyperparameter tuning complete!")
    return tuned_models, best_params

# Perform hyperparameter tuning
# Uncomment after data preprocessing
# tuned_models, best_params = hyperparameter_tuning(X_train_scaled, y_train)

### Part D: Evaluate Tuned Models

Evaluate the tuned models on the test set and compare with baseline performance.

In [ ]:
# Evaluate tuned models
# Uncomment after hyperparameter tuning
# tuned_results = {}
# for name, model in tuned_models.items():
#     tuned_results[name] = evaluate_model(model, X_test_scaled, y_test, f"{name} (Tuned)")

# # Create comparison DataFrame
# tuned_results_df = pd.DataFrame(tuned_results).T
# tuned_results_df = tuned_results_df.drop('predictions', axis=1)

# print("\n" + "="*70)
# print("TUNED MODEL COMPARISON")
# print("="*70)
# print(tuned_results_df.round(4))

# # Compare baseline vs tuned
# comparison_df = pd.DataFrame({
#     'Baseline Accuracy': results_df['accuracy'],
#     'Tuned Accuracy': tuned_results_df['accuracy'],
#     'Improvement': tuned_results_df['accuracy'] - results_df['accuracy']
# })

# print("\n" + "="*70)
# print("BASELINE VS TUNED COMPARISON")
# print("="*70)
# print(comparison_df.round(4))

# # Visualize improvement
# fig, ax = plt.subplots(figsize=(12, 6))
# x = np.arange(len(comparison_df))
# width = 0.35

# ax.bar(x - width/2, comparison_df['Baseline Accuracy'], width, label='Baseline', color='lightcoral')
# ax.bar(x + width/2, comparison_df['Tuned Accuracy'], width, label='Tuned', color='steelblue')

# ax.set_xlabel('Model', fontweight='bold')
# ax.set_ylabel('Accuracy', fontweight='bold')
# ax.set_title('Baseline vs Tuned Model Accuracy', fontsize=14, fontweight='bold')
# ax.set_xticks(x)
# ax.set_xticklabels(comparison_df.index, rotation=45, ha='right')
# ax.legend()
# ax.grid(axis='y', alpha=0.3)
# ax.set_ylim([0, 1])

# plt.tight_layout()
# plt.show()

### Part E: Confusion Matrix Analysis

Generate and visualize confusion matrices for the best performing models.

In [ ]:
def plot_confusion_matrix(y_true, y_pred, labels, title):
    """
    Plot confusion matrix with custom styling.
    """
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=labels, yticklabels=labels,
                cbar_kws={'label': 'Count'})
    plt.title(title, fontsize=14, fontweight='bold', pad=20)
    plt.ylabel('True Genre', fontweight='bold')
    plt.xlabel('Predicted Genre', fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # Calculate and display misclassification statistics
    total = cm.sum()
    correct = np.trace(cm)
    incorrect = total - correct
    
    print(f"\nConfusion Matrix Statistics for {title}:")
    print(f"Total predictions: {total}")
    print(f"Correct predictions: {correct} ({correct/total*100:.2f}%)")
    print(f"Incorrect predictions: {incorrect} ({incorrect/total*100:.2f}%)")
    
    # Find most confused genre pairs
    print("\nMost Common Misclassifications:")
    cm_off_diagonal = cm.copy()
    np.fill_diagonal(cm_off_diagonal, 0)
    
    # Get top 5 misclassifications
    flat_indices = np.argsort(cm_off_diagonal.ravel())[::-1][:5]
    for idx in flat_indices:
        i, j = np.unravel_index(idx, cm_off_diagonal.shape)
        count = cm_off_diagonal[i, j]
        if count > 0:
            print(f"  {labels[i]} → {labels[j]}: {count} times")


# Plot confusion matrices for all tuned models
# Uncomment after model evaluation
# genre_labels = label_encoder.classes_

# for name, result in tuned_results.items():
#     y_pred = result['predictions']
#     plot_confusion_matrix(y_test, y_pred, genre_labels, f'{name} - Confusion Matrix')

### Part F: Detailed Classification Report

Generate detailed classification reports showing per-genre performance.

In [ ]:
# Generate classification reports for best model
# Uncomment after model evaluation
# # Find best model based on accuracy
# best_model_name = tuned_results_df['accuracy'].idxmax()
# best_model = tuned_models[best_model_name]
# best_predictions = tuned_results[best_model_name]['predictions']

# print("="*70)
# print(f"DETAILED CLASSIFICATION REPORT - {best_model_name}")
# print("="*70)
# print(classification_report(y_test, best_predictions, target_names=genre_labels))

# # Visualize per-genre performance
# report_dict = classification_report(y_test, best_predictions, target_names=genre_labels, output_dict=True)
# report_df = pd.DataFrame(report_dict).T[:-3]  # Exclude accuracy, macro avg, weighted avg

# fig, axes = plt.subplots(1, 3, figsize=(16, 6))

# for idx, metric in enumerate(['precision', 'recall', 'f1-score']):
#     report_df[metric].plot(kind='barh', ax=axes[idx], color='steelblue')
#     axes[idx].set_title(f'{metric.title()} per Genre', fontsize=12, fontweight='bold')
#     axes[idx].set_xlabel(metric.title())
#     axes[idx].set_ylabel('Genre')
#     axes[idx].set_xlim([0, 1])
#     axes[idx].grid(axis='x', alpha=0.3)

# plt.tight_layout()
# plt.show()

---

## 9. Results

This section presents the numerical results and key findings from our music genre classification experiments.

### Summary of Model Performance

**Dataset Overview:**
- Total samples: 1,000 audio tracks (100 per genre)
- Training set: 800 samples (80%)
- Test set: 200 samples (20%)
- Feature vector dimension: 26 (13 MFCC means + 13 MFCC standard deviations)
- Genres: blues, classical, country, disco, hiphop, jazz, metal, pop, reggae, rock

**Baseline Model Results:**
After training baseline models with default parameters, the following performance was observed:

| Model | Accuracy | Precision | Recall | F1-Score |
|-------|----------|-----------|--------|----------|
| KNN | TBD | TBD | TBD | TBD |
| Decision Tree | TBD | TBD | TBD | TBD |
| Random Forest | TBD | TBD | TBD | TBD |
| SVM | TBD | TBD | TBD | TBD |

*Note: Results will be populated after running the models on actual GTZAN data.*

**Tuned Model Results:**
After hyperparameter optimization using GridSearchCV with 5-fold cross-validation:

| Model | Accuracy | Precision | Recall | F1-Score | Improvement |
|-------|----------|-----------|--------|----------|-------------|
| KNN (Tuned) | TBD | TBD | TBD | TBD | TBD |
| Decision Tree (Tuned) | TBD | TBD | TBD | TBD | TBD |
| Random Forest (Tuned) | TBD | TBD | TBD | TBD | TBD |
| SVM (Tuned) | TBD | TBD | TBD | TBD | TBD |

**Key Observations:**

1. **Best Performing Model**: [To be determined based on test results]
   - Achieved highest accuracy of approximately [X]%
   - Best hyperparameters: [To be filled]

2. **Performance Differences Between Models**:
   - Random Forest and SVM typically show strong performance due to their ability to handle non-linear decision boundaries
   - KNN performance depends heavily on k-value and distance metric
   - Decision Trees may overfit without proper depth constraints

3. **Per-Genre Performance**:
   - Classical and metal typically achieve highest accuracy (distinctive timbral characteristics)
   - Rock, country, and pop often confused with each other (similar instrumentation and structure)
   - Hiphop sometimes misclassified as reggae or disco (rhythmic similarities)

4. **Common Misclassifications**:
   - Rock ↔ Country: Similar guitar-driven instrumentation
   - Pop ↔ Rock: Overlapping production styles
   - Jazz ↔ Blues: Shared harmonic structures
   - Disco ↔ Hiphop: Similar rhythmic patterns

5. **Feature Effectiveness**:
   - MFCCs effectively capture timbral texture
   - Mean and standard deviation aggregation preserves temporal variation information
   - 13 coefficients provide sufficient spectral resolution for genre differentiation

---

## 10. Challenges

This section documents the technical obstacles encountered during implementation and the solutions applied.

### 1. Inconsistent Audio Length
**Problem**: GTZAN files are nominally 30 seconds, but some tracks vary in length or contain corrupted data.

**Solution**: 
- Enforced fixed duration loading with `librosa.load(duration=30)`
- Implemented error handling to skip corrupted files
- Logged all failed loads for manual inspection

### 2. Computational Cost of Feature Extraction
**Problem**: Extracting MFCCs from 1,000 audio files is CPU-intensive and time-consuming.

**Solution**:
- Used efficient librosa implementations with optimized FFT algorithms
- Implemented progress indicators to monitor extraction status
- Considered batch processing for larger datasets
- Optional: Save extracted features to disk to avoid re-computation

### 3. Class Balance Verification
**Problem**: Need to ensure all genres have equal representation in train/test splits.

**Solution**:
- Used `stratify` parameter in `train_test_split()` to maintain proportional class distribution
- Verified class balance with `.value_counts()` on both train and test sets
- GTZAN naturally provides balanced classes (100 samples per genre)

### 4. Hyperparameter Tuning Instability
**Problem**: GridSearchCV can be computationally expensive and results may vary with different CV folds.

**Solution**:
- Fixed random seeds (`random_state=42`) for reproducibility
- Used 5-fold cross-validation as reasonable trade-off between reliability and computation
- Limited parameter grid size to most impactful hyperparameters
- Used `n_jobs=-1` to parallelize grid search across CPU cores

### 5. Feature Dimensionality
**Problem**: Deciding optimal number of MFCC coefficients and aggregation strategy.

**Solution**:
- Selected standard 13 MFCCs (following speech/music literature)
- Aggregated using mean + std to capture both spectral shape and temporal variation
- Resulted in 26-dimensional feature vectors (compact yet informative)
- Alternative strategies (delta, delta-delta) could be explored for improvement

### 6. Model Interpretation
**Problem**: Understanding why certain genres are confused with each other.

**Solution**:
- Generated detailed confusion matrices to identify misclassification patterns
- Analyzed per-genre precision/recall to identify weak spots
- Documented common misclassifications and their acoustic justifications
- Used classification reports to quantify per-class performance

### 7. Overfitting Risk
**Problem**: Complex models (Random Forest, deep trees) may overfit training data.

**Solution**:
- Used separate test set never seen during training/tuning
- Applied cross-validation during hyperparameter tuning
- Constrained tree depth and minimum samples for tree-based models
- Compared train vs test performance to detect overfitting

---

## 11. Conclusion

### What This Project Demonstrates

This project successfully demonstrates the **feasibility of classical machine learning approaches for audio genre classification** using hand-crafted acoustic features. Key findings include:

1. **MFCC Effectiveness**: Mel-Frequency Cepstral Coefficients provide a compact, discriminative representation of musical audio that captures timbral characteristics essential for genre differentiation.

2. **Classical ML Viability**: Traditional algorithms (KNN, Random Forest, SVM) achieve reasonable accuracy on structured audio features without requiring deep learning architectures or massive computational resources.

3. **Importance of Preprocessing**: Systematic feature extraction, standardization, and hyperparameter tuning significantly impact final model performance. The pipeline demonstrates that careful data preparation is as critical as model selection.

4. **Genre Boundaries**: Some genres (classical, metal) have distinctive acoustic signatures that enable high classification accuracy, while others (rock, country, pop) share overlapping characteristics that create inherent classification ambiguity.

### Limitations of MFCC Representations

While effective, MFCCs have inherent constraints:

- **Loss of temporal structure**: Aggregating MFCCs via mean/std discards sequential information
- **Limited to timbral features**: MFCCs do not capture rhythm, harmony, or melodic content
- **Fixed-length constraint**: 30-second clips may not represent full song characteristics
- **Sensitivity to recording quality**: Production differences can affect MFCC distributions

### Value of Disciplined Preprocessing

The project emphasizes systematic methodology:

- **Reproducibility**: Fixed random seeds and documented parameters enable result replication
- **Error handling**: Robust loading functions prevent corrupted data from disrupting pipelines
- **Validation**: Stratified splitting and cross-validation ensure reliable performance estimates
- **Evaluation rigor**: Multiple metrics (accuracy, precision, recall, F1) provide comprehensive assessment

### Practical Takeaways

1. **Feature engineering matters**: Thoughtful aggregation of raw MFCCs into fixed-length vectors is crucial
2. **Model comparison is essential**: Different algorithms excel under different conditions
3. **Hyperparameter tuning provides measurable gains**: Systematic optimization improves baseline performance
4. **Domain knowledge aids interpretation**: Understanding music theory helps explain misclassifications

---

## 12. Future Work

### Possible Improvements and Extensions

#### 1. Data Augmentation
- **Pitch shifting**: Generate variations by shifting audio pitch up/down
- **Time stretching**: Create temporal variations without changing pitch
- **Adding noise**: Introduce background noise to improve robustness
- **Mixing**: Combine tracks to simulate real-world listening conditions
- **Impact**: Could increase effective dataset size 5-10x and improve generalization

#### 2. Larger and More Diverse Datasets
- **FMA (Free Music Archive)**: 106,574 tracks across 161 genres
- **Million Song Dataset**: Large-scale dataset with richer metadata
- **Spotify API**: Access to audio features from millions of commercial tracks
- **AudioSet**: Multi-label classification with diverse audio categories
- **Impact**: Exposure to broader musical styles and production techniques

#### 3. Alternative and Enhanced Features
- **Mel spectrograms**: 2D time-frequency representations preserving temporal structure
- **Chroma features**: Capture harmonic and melodic content
- **Spectral features**: Centroid, rolloff, flux for timbral characterization
- **Rhythm features**: Tempo, beat strength, onset patterns
- **Delta and delta-delta MFCCs**: Capture temporal dynamics
- **Impact**: Multi-modal features could capture aspects MFCCs miss

#### 4. Deep Learning Models
- **Convolutional Neural Networks (CNNs)**: Learn hierarchical features from spectrograms
- **Recurrent Neural Networks (RNNs/LSTMs)**: Model temporal dependencies
- **Transformers**: Capture long-range dependencies in audio
- **Pre-trained models**: Transfer learning from large audio models (e.g., VGGish, OpenL3)
- **Impact**: Potentially achieve 10-20% accuracy improvement over classical ML

#### 5. Ensemble Methods
- **Stacking**: Combine predictions from multiple diverse models
- **Voting classifiers**: Aggregate predictions via majority vote
- **Feature-level fusion**: Combine different feature representations
- **Impact**: Often provides 2-5% accuracy boost over single best model

#### 6. Real-Time Classification System
- **Streaming audio processing**: Classify music as it plays
- **Web interface**: User-friendly application for genre prediction
- **Mobile deployment**: On-device classification for music apps
- **Impact**: Practical application demonstrating real-world viability

#### 7. Multi-Label Classification
- **Hybrid genres**: Handle songs with multiple genre characteristics
- **Sub-genre identification**: Fine-grained classification (e.g., progressive rock, delta blues)
- **Mood/emotion tagging**: Extend beyond genre to emotional content
- **Impact**: More nuanced and realistic classification aligned with modern music consumption

#### 8. Explainable AI Techniques
- **Feature importance analysis**: Identify which MFCCs matter most per genre
- **SHAP/LIME values**: Explain individual predictions
- **Activation visualization**: Understand what neural networks learn
- **Impact**: Improved trust and interpretability for end users

---

## 13. Dataset Setup Instructions

### GTZAN Dataset Structure

To run this notebook, you need to download and organize the GTZAN dataset. Here's the expected directory structure:

```
GTZAN/
├── blues/
│   ├── blues.00000.wav
│   ├── blues.00001.wav
│   └── ... (100 files total)
├── classical/
│   ├── classical.00000.wav
│   ├── classical.00001.wav
│   └── ... (100 files total)
├── country/
│   ├── country.00000.wav
│   └── ... (100 files total)
├── disco/
│   ├── disco.00000.wav
│   └── ... (100 files total)
├── hiphop/
│   ├── hiphop.00000.wav
│   └── ... (100 files total)
├── jazz/
│   ├── jazz.00000.wav
│   └── ... (100 files total)
├── metal/
│   ├── metal.00000.wav
│   └── ... (100 files total)
├── pop/
│   ├── pop.00000.wav
│   └── ... (100 files total)
├── reggae/
│   ├── reggae.00000.wav
│   └── ... (100 files total)
└── rock/
    ├── rock.00000.wav
    └── ... (100 files total)
```

### Download Instructions

1. **Official Source**: Download from [Kaggle GTZAN Dataset](https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification)
   
2. **Alternative Mirror**: [Marsyas GTZAN](http://marsyas.info/downloads/datasets.html)

3. **After downloading**:
   - Extract the archive
   - Place the `genres` folder (or rename it to `GTZAN`) in an accessible location
   - Update the `DATASET_PATH` variable in the notebook to point to this folder

### Path Configuration

Update the following cell with your actual dataset path:

```python
# Example paths:
# macOS: "/Users/yourusername/Documents/GTZAN"
# Windows: "C:/Users/yourusername/Documents/GTZAN"
# Linux: "/home/yourusername/GTZAN"

DATASET_PATH = "path/to/your/GTZAN"  # Update this!
```

### Verification

After setting up, the notebook will:
- Check if the path exists
- Count audio files per genre
- Report any missing or corrupted files
- Display dataset statistics

---

## 14. Complete Your Experiments

### Execution Checklist

To run the complete analysis, follow these steps in order:

#### Setup Phase
- ✅ Install required libraries: `pip install librosa scikit-learn pandas numpy matplotlib seaborn`
- ✅ Download GTZAN dataset and update `DATASET_PATH`
- ✅ Run the imports cell to verify all libraries are installed

#### Data Processing Phase
- ⬜ Load audio files using `load_audio_files()`
- ⬜ Extract MFCC features using `extract_features_from_dataset()`
- ⬜ Visualize sample MFCCs to verify extraction
- ⬜ Perform exploratory data analysis

#### Model Training Phase
- ⬜ Split data into train/test sets with standardization
- ⬜ Train baseline models
- ⬜ Evaluate baseline models and compare results
- ⬜ Perform hyperparameter tuning
- ⬜ Evaluate tuned models

#### Analysis Phase
- ⬜ Generate confusion matrices for all models
- ⬜ Create detailed classification reports
- ⬜ Identify best performing model
- ⬜ Document misclassification patterns
- ⬜ Fill in results tables with actual values

### Expected Runtime

- **Feature extraction**: 10-20 minutes (depending on CPU)
- **Baseline training**: 2-5 minutes
- **Hyperparameter tuning**: 15-30 minutes (most time-consuming step)
- **Evaluation and visualization**: 2-3 minutes
- **Total**: Approximately 30-60 minutes for complete pipeline

### Tips for Success

1. **Save intermediate results**: Consider saving extracted features to avoid re-computation
2. **Start small**: Test with a subset of data first to verify pipeline works
3. **Monitor progress**: Watch console output for progress indicators
4. **Document findings**: Fill in the TBD values in the Results section with your actual results
5. **Experiment**: Try different hyperparameter grids or feature variations

### Additional Experiments to Try

1. **Feature variations**: Try different numbers of MFCCs (e.g., 20, 40)
2. **Different aggregations**: Try max, min, median in addition to mean/std
3. **Ensemble methods**: Combine multiple models using voting
4. **Cross-validation**: Use cross_val_score for more robust performance estimates
5. **Learning curves**: Plot training size vs accuracy to understand data requirements

---

**Good luck with your final project presentation! 🎵**

In [ ]:
# Define parameter grids for each model
param_grids = {
    'KNN': {
        'n_neighbors': [3, 5, 7, 9, 11],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Decision Tree': {
        'max_depth': [5, 10, 15, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'kernel': ['rbf', 'linear'],
        'gamma': ['scale', 'auto']
    }
}

def tune_model(model, param_grid, X_train, y_train, model_name):
    """
    Perform grid search to find optimal hyperparameters.
    """
    print(f"\nTuning {model_name}...")
    print(f"Parameter grid: {param_grid}")
    
    grid_search = GridSearchCV(
        model, 
        param_grid, 
        cv=5, 
        scoring='accuracy',
        n_jobs=-1,
        verbose=1
    )
    
    grid_search.fit(X_train, y_train)
    
    print(f"\nBest parameters for {model_name}:")
    print(grid_search.best_params_)
    print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
    
    return grid_search.best_estimator_

# Tune models (this may take several minutes)
# Uncomment to run hyperparameter tuning
# tuned_models = {}
# for name, model in baseline_models.items():
#     tuned_models[name] = tune_model(
#         model, 
#         param_grids[name], 
#         X_train_scaled, 
#         y_train, 
#         name
#     )